In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 38.8 MB/s eta 0:00:00


In [3]:
import csv
import time
import json
import os
import pandas as pd
import tqdm
import numpy

from implicit.nearest_neighbours import bm25_weight
from implicit.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix

/usr/local/lib/python3.8/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


# Below is the Final csv file with listening playcount of each user per artist.

In [4]:
DATA_ROOT = '/content/drive/MyDrive/amplab/assignment-1/' # mention path to your folder containing the user-artist playcount file
FINAL_FILE = os.path.join(DATA_ROOT,'final6-playcount.csv')

In [5]:
# read the csv file and output a dataframe
df = pd.read_csv(FINAL_FILE)
df.head()

,user_id,artist_mbids,artist_name,play_count
0,8510,69737ebe-836c-40dc-a938-7e85688426ff,Alain Souchon,2
1,300,27e2997f-f7a1-4353-bcc4-57b9274fa9a4,BABYMETAL,1
2,10012,d724e786-2385-4244-80a5-b71724b714e7,Screamer,14
3,18600,43c0958c-27ef-46c4-b4cc-83cfca54170d,The Devil Wears Prada,1
4,19725,ab1a3f85-e0ea-470a-af5c-175447ae774c,underscores,1


In [6]:
df1 = df.copy()
df1.head(100)

,user_id,artist_mbids,artist_name,play_count
0,8510,69737ebe-836c-40dc-a938-7e85688426ff,Alain Souchon,2
1,300,27e2997f-f7a1-4353-bcc4-57b9274fa9a4,BABYMETAL,1
2,10012,d724e786-2385-4244-80a5-b71724b714e7,Screamer,14
3,18600,43c0958c-27ef-46c4-b4cc-83cfca54170d,The Devil Wears Prada,1
4,19725,ab1a3f85-e0ea-470a-af5c-175447ae774c,underscores,1
...,...,...,...,...
95,3802,a8e935c6-3fcc-414c-900c-77e8170e7e7c,Black Label Society,20
96,19351,95bb27d2-43c7-4a92-85ee-f95a245f0ee4,John Fahey,1
97,20848,724ffc08-6987-40b0-b107-03bb95a82b80,Deorro,1
98,1282,51658fa8-672b-450f-b245-489942670631,Giampi Daldello,2


In [7]:
df1['artist_name'] = df['artist_name'].astype('category')

# get the categorical codes as an array
codes = df1['artist_name'].cat.codes.values
print(codes) # Output: [0 1 2]

# get the categories as an array
categories = df1['artist_name'].cat.categories.values
print(categories) # Output: ['Beatles' 'LedZepplin' 'LinkinPark']


[  7655  20074 197275 ... 220176 174349 103549]
['!!!' '!Action Pact!' '!Attention!' ... '🌴1am バイブBlunts' '👁\u200d🗨📲' '🧫']


In [8]:
df1['artist_name'] = df1['artist_name'].cat.codes
df1.head()

,user_id,artist_mbids,artist_name,play_count
0,8510,69737ebe-836c-40dc-a938-7e85688426ff,7655,2
1,300,27e2997f-f7a1-4353-bcc4-57b9274fa9a4,20074,1
2,10012,d724e786-2385-4244-80a5-b71724b714e7,197275,14
3,18600,43c0958c-27ef-46c4-b4cc-83cfca54170d,223360,1
4,19725,ab1a3f85-e0ea-470a-af5c-175447ae774c,263872,1


In [9]:
# Convert your DataFrame to a CSR sparse matrix
sparse_matrix = csr_matrix((df1['play_count'], (df1['user_id'], df1['artist_name'])))
print(sparse_matrix)

  (1, 486)	1
  (1, 774)	1
  (1, 1085)	8
  (1, 1331)	1
  (1, 1426)	1
  (1, 1586)	1
  (1, 2025)	3
  (1, 2108)	13
  (1, 2256)	1
  (1, 2399)	9
  (1, 2405)	1
  (1, 2648)	1
  (1, 2881)	19
  (1, 3005)	8
  (1, 3008)	2
  (1, 3723)	18
  (1, 4035)	325
  (1, 5791)	82
  (1, 6030)	1
  (1, 6053)	3
  (1, 6092)	55
  (1, 6374)	1
  (1, 6562)	76
  (1, 6885)	1
  (1, 6941)	40
  :	:
  (21562, 276076)	8
  (21562, 276116)	3
  (21562, 276146)	6
  (21562, 276252)	1
  (21562, 276342)	6
  (21562, 276788)	1
  (21562, 277096)	7
  (21562, 277294)	4
  (21562, 277339)	1
  (21562, 277352)	1
  (21562, 277427)	395
  (21562, 277436)	49
  (21562, 277507)	41
  (21562, 277587)	8
  (21562, 277700)	5
  (21562, 277864)	12
  (21562, 278117)	8
  (21562, 278241)	2
  (21562, 278707)	2
  (21562, 278920)	7
  (21562, 279135)	1
  (21562, 279247)	20
  (21562, 279846)	1
  (21562, 279914)	31
  (21562, 280102)	13


In [10]:
bob_rows = df1.loc[(df['user_id'] == 8510) & (df1['artist_name'] == 7655)]
bob_rows

,user_id,artist_mbids,artist_name,play_count
0,8510,69737ebe-836c-40dc-a938-7e85688426ff,7655,2


In [35]:
sparse_matrix.shape

(21563, 280252)

In [27]:
from implicit.nearest_neighbours import bm25_weight

# weight the matrix, both to reduce impact of users that have played the same artist thousands of times
# and to reduce the weight given to popular items
sparse_matrix = bm25_weight(sparse_matrix, K1=100, B=0.8)

In [28]:
model = AlternatingLeastSquares(factors=50, regularization=0.01)
model.fit(sparse_matrix)


/usr/local/lib/python3.8/dist-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.035865068435668945 seconds
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

In [29]:
# get recommendations for user 1
user_id = 1
# user_items = data_sparse[user_id].toarray().flatten()
artist_name_codes , scores = model.recommend(user_id, sparse_matrix, filter_already_liked_items=False)
# find related items
similar_artists, similarity_scores = model.similar_items(artist_name_codes[0])


In [30]:
df['artist_name'] = df['artist_name'].astype('category')

# get the categorical codes as an array
codes = df['artist_name'].cat.codes.values
print(codes) # Output: [0 1 1 2 0]

# get the categories as an array
categories = df['artist_name'].cat.categories.values

[  7655  20074 197275 ... 220176 174349 103549]


In [31]:
original_names = categories[artist_name_codes]
original_names

array(['Thievery Corporation', 'Zero 7', 'Bonobo', 'Massive Attack',
       'Jean‐Michel Jarre', 'GusGus', 'Portishead', 'Boards of Canada',
       'Morcheeba', 'Groove Armada'], dtype=object)

In [32]:
pd.DataFrame({"artist": original_names, "score": scores})

,artist,score
0,Thievery Corporation,1.244949
1,Zero 7,1.194673
2,Bonobo,1.189922
3,Massive Attack,1.174771
4,Jean‐Michel Jarre,1.170155
5,GusGus,1.168698
6,Portishead,1.167202
7,Boards of Canada,1.161827
8,Morcheeba,1.152229
9,Groove Armada,1.150235


In [21]:
pd.DataFrame({"artist": original_names, "score": scores})

,artist,score
0,GusGus,1.304948
1,Faithless,1.279368
2,Trentemøller,1.277576
3,Archive,1.248061
4,Pet Shop Boys,1.243270
5,a‐ha,1.201715
6,Moby,1.198578
7,Tears for Fears,1.197010
8,Bluetech,1.193352
9,Parov Stelar,1.192095


In [19]:
pd.DataFrame({"artist": original_names, "score": scores})

,artist,score
0,GusGus,1.304948
1,Faithless,1.279368
2,Trentemøller,1.277576
3,Archive,1.248061
4,Pet Shop Boys,1.243270
5,a‐ha,1.201715
6,Moby,1.198578
7,Tears for Fears,1.197010
8,Bluetech,1.193352
9,Parov Stelar,1.192095


In [33]:
print(similar_artists)
print(similarity_scores)

similar_artists = categories[similar_artists]


[234923   6941 161765 145148 178017 257485  91147 152593  71361  30881]
[0.9999999  0.93484396 0.9275575  0.925417   0.91616935 0.91567093
 0.9145763  0.9062055  0.9051323  0.9022717 ]


In [34]:
pd.DataFrame({"artist": similar_artists, "score": similarity_scores})

,artist,score
0,Thievery Corporation,1.000000
1,Air,0.934844
2,Nightmares on Wax,0.927558
3,Massive Attack,0.925417
4,Portishead,0.916169
5,Zero 7,0.915671
6,Groove Armada,0.914576
7,Moby,0.906205
8,Emancipator,0.905132
9,Bonobo,0.902272


In [23]:
pd.DataFrame({"artist": similar_artists, "score": similarity_scores})

,artist,score
0,Faithless,1.000000
1,Underworld,0.877961
2,Groove Armada,0.867140
3,The Chemical Brothers,0.860397
4,Leftfield,0.859982
5,Fatboy Slim,0.856547
6,Röyksopp,0.853488
7,William Orbit,0.846664
8,Kosheen,0.843070
9,Delerium,0.842721


In [26]:
pd.DataFrame({"artist": similar_artists, "score": similarity_scores})

,artist,score
0,Archive,1.000000
1,Trentemøller,0.845030
2,Terranova,0.832423
3,Lamb,0.826686
4,Troy Bonnes,0.811245
5,Max Vertigo,0.809702
6,Hooverphonic,0.805015
7,The Cinematic Orchestra,0.803629
8,A&D,0.802002
9,HÆLOS,0.798122
